In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from bart_playground import DefaultBART, Tree, DataGenerator
import bartz

In [3]:
proposal_probs = {"grow" : 0.4,
                  "prune" : 0.4,
                  "change" : 0.1,
                  "swap" : 0.1}
generator = DataGenerator(n_samples=1000, n_features=2, noise=0.1, random_seed=42)
X, y = generator.generate(scenario="piecewise_flat")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
np.set_printoptions(suppress=True)
print(y_train[:12])

[ 0.59965549  0.66000473  0.60195109  0.59759841 -0.48024437 -0.05465378
 -0.09076175 -0.06775115  0.44628062  0.51643207 -0.17564767 -0.18611937]


In [4]:
# initialize numba
bart = DefaultBART(ndpost=200, nskip=100, n_trees=100, proposal_probs=proposal_probs)
bart.fit(X_train, y_train)

Iterations: 100%|██████████| 300/300 [00:04<00:00, 62.92it/s]


In [5]:
import pandas as pd

# Collect move counts
selected = bart.sampler.move_selected_counts
success = bart.sampler.move_success_counts
accepted = bart.sampler.move_accepted_counts

# Combine into a DataFrame for easy viewing
df = pd.DataFrame({
    "selected": pd.Series(selected),
    "success": pd.Series(success),
    "accepted": pd.Series(accepted)
})

# Add success, acceptance and change rates
df["success_rate"] = df["success"] / df["selected"]
df["accept_rate"] = df["accepted"] / df["success"] # The "acceptance rate" reflects the MH acceptance probability
df["change_rate"] = df["accepted"] / df["selected"] # The "change rate" reflects the frequency of tree changes

print(df)

        selected  success  accepted  success_rate  accept_rate  change_rate
grow       11825    11825      3621      1.000000     0.306216     0.306216
prune      12191    11493      3471      0.942745     0.302010     0.284718
change      3014     2839      1019      0.941938     0.358929     0.338089
swap        2970     1012       377      0.340741     0.372530     0.126936


In [6]:
bart2 = DefaultBART(ndpost=400, nskip=100, n_trees=100, proposal_probs=proposal_probs)
%prun -s cumtime -D profile_bart.prof -q bart2.fit(X_train, y_train)
!gprof2dot -f pstats profile_bart.prof -o profile_bart.dot
!dot -Tpng profile_bart.dot -o profile_bart.png

Iterations:   0%|          | 0/500 [00:00<?, ?it/s]

Iterations: 100%|██████████| 500/500 [00:10<00:00, 45.77it/s]


 
*** Profile stats marshalled to file 'profile_bart.prof'.


In [7]:
arrays = [tree.vars for tree in bart.trace[-1].trees]
counts = np.array([np.count_nonzero(arr >= 0) for arr in arrays])
print(counts)
deep_trees = np.array([count >= 3 for count in counts])
print(np.where(deep_trees))

[1 3 1 3 2 2 3 1 1 2 0 1 2 2 3 2 1 1 2 2 0 3 4 2 2 2 1 1 1 2 2 1 1 0 1 1 1
 2 0 1 1 3 1 2 1 2 1 2 3 5 3 1 1 1 1 1 2 1 2 1 3 1 2 1 0 1 2 1 1 0 0 1 2 2
 1 0 2 2 1 1 1 1 2 1 1 1 5 1 2 1 0 1 0 1 1 2 2 2 2 1]
(array([ 1,  3,  6, 14, 21, 22, 41, 48, 49, 50, 60, 86]),)


In [8]:
np.mean([len(arr) for arr in arrays])

np.float64(8.8)

In [9]:
print(bart.sampler.trace[-1].global_params)

{'eps_sigma2': array([0.00642036])}


In [10]:
from bart_playground import visualize_tree
tree_sp : Tree = bart.sampler.trace[-1].trees[50]

print(tree_sp)
print(tree_sp.vars)
print(tree_sp.leaf_vals)

X_0 <= 0.763697386 (split, n = 750)
	X_1 <= 0.514178932 (split, n = 562)
		Val: -0.000567809 (leaf, n = 294)
		Val: 0.024965059 (leaf, n = 268)
	X_1 <= 0.092173070 (split, n = 188)
		Val: 0.014175317 (leaf, n = 8)
		Val: -0.013555662 (leaf, n = 180)
[ 0  1  1 -1 -1 -1 -1 -2]
[        nan         nan         nan -0.00056781  0.02496506  0.01417532
 -0.01355566         nan]


In [11]:
rf = RandomForestRegressor(random_state=42)
lr = LinearRegression()
rf.fit(X_train, y_train)
lr.fit(X_train, y_train)

btz = bartz.BART.gbart(np.transpose(X_train), y_train, ntree=100, ndpost=200, nskip=100)
btpred_all = btz.predict(np.transpose(X_test))
btpred = np.mean(np.array(btpred_all), axis=0)

INFO:2025-09-09 20:41:24,802:jax._src.xla_bridge:867: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


Iteration 100/300 P_grow=0.57 P_prune=0.43 A_grow=0.28 A_prune=0.40 (burnin)
Iteration 200/300 P_grow=0.54 P_prune=0.46 A_grow=0.30 A_prune=0.24
Iteration 300/300 P_grow=0.52 P_prune=0.48 A_grow=0.25 A_prune=0.31


In [12]:
models = {"bart" : bart, 
          "rf" : rf, 
          "lr" : lr,
          "btz" : btz}
results = {}
for model_name, model in models.items():
    if model_name == "btz":
        results[model_name] = mean_squared_error(y_test, btpred)
    else:
        results[model_name] = mean_squared_error(y_test, model.predict(X_test))
results

{'bart': 0.016913205213783065,
 'rf': 0.01570244066836391,
 'lr': 0.04058944845865094,
 'btz': 0.01451765109911874}

In [13]:
print(bart.sampler.trace[-1].evaluate(X_train)[:12])
print(bart.preprocessor.transform_y(y_train)[:12])

[ 0.31316486  0.28736398  0.31458274  0.30757785 -0.39123347  0.07929082
 -0.02212593  0.07817362  0.2402342   0.34557924 -0.09955953  0.03123178]
[ 0.36207604  0.4024236   0.3636108   0.36070073 -0.35991025 -0.07537408
 -0.09951471 -0.08413056  0.25953454  0.30643553 -0.1562667  -0.16326775]


In [14]:
mean_squared_error(y_test, np.ones_like(y_test) * y_test.mean())

0.08625514042334204

In [15]:
if all([(bart.sampler.trace[-1].trees[i].evaluate() == bart.sampler.trace[-1].trees[i].evaluate(X_train)).all()
            for i in range(100)]):
    print("True")
else:
    print("False")

True


In [16]:
if np.allclose(bart.trace[-1].evaluate(), bart.trace[-1].evaluate(X_train), atol=1e-6):
    print("True")
else:
    print("False")

True
